In [ ]:
import pandas as pdas  # CSV files In/Output, data processing (e.g. pdas.read_csv)
import numpy as npy  # Evaluating mathematical lib, linear algebra
import matplotlib.pyplot as matplot_plt  # Draw chart lib
import plotly.express as plotly_ex  # Draw chart lib

# For Saving best model
import joblib

In [ ]:
# Auto detect-width to set data to fit the width of the terminal
pdas.set_option("display.max_columns", None)
# Input the dataset
dts_dtframe = pdas.read_csv("../data/churn.csv")

In [ ]:
# Get overview of the data
def dataoveriew(dataframe, message):
    print(f"{message}:")
    print("Tổng số hàng: ", dataframe.shape[0])
    print("Tổng số thuộc tính:", dataframe.shape[1])
    print("Các thuộc tính:")
    print(dataframe.columns.tolist())
    print("Số kiểu giá trị của từng thuộc tính:")
    print(dataframe.nunique())
    print("Giá trị null:", dataframe.isnull().sum().values.sum())


dataoveriew(dts_dtframe, "Tổng quan về dataset:")

### Explore Target variable

In [ ]:
# Return the Object in descending order containing counts of unique values
churn_instance = dts_dtframe["Churn"].value_counts()
# Convert series, object to DataFrame
churn_instance = churn_instance.to_frame()
# Reset index of the DataFrame
churn_instance = churn_instance.reset_index()
# Rename the column 'index'
churn_instance = churn_instance.rename(columns={"index": "Category"})
# Use plotly.express lib to draw the chart
churn_pie_chart = plotly_ex.pie(
    churn_instance,
    values="Churn",
    names="Category",
    color_discrete_sequence=["blue", "red"],
    title="Distribution of Churn",
)
churn_pie_chart.show()

### Explore Categorical features

In [ ]:
# Defining bar chart function
def bar_chart(feature, df=dts_dtframe):
    # Groupby the categorical feature
    temp_df = df.groupby([feature, "Churn"]).size().reset_index()
    temp_df = temp_df.rename(columns={0: "Count"})
    # Calculate the value counts of each distribution and it's corresponding Percentages
    value_counts_df = df[feature].value_counts().to_frame().reset_index()
    categories = [cat[1][0] for cat in value_counts_df.iterrows()]
    # Calculate the value counts of each distribution and it's corresponding Percentages
    num_list = [num[1][1] for num in value_counts_df.iterrows()]
    div_list = [element / sum(num_list) for element in num_list]
    percentage = [round(element * 100, 1) for element in div_list]
    # Defining string formatting for graph annotation
    # Numeric section
    def num_format(list_instance):
        formatted_str = ""
        for index, num in enumerate(list_instance):
            if index < len(list_instance) - 2:
                formatted_str = (
                    formatted_str + f"{num}%, "
                )  # append to empty string(formatted_str)
            elif index == len(list_instance) - 2:
                formatted_str = formatted_str + f"{num}% & "
            else:
                formatted_str = formatted_str + f"{num}%"
        return formatted_str

    # Categorical section
    def str_format(list_instance):
        formatted_str = ""
        for index, cat in enumerate(list_instance):
            if index < len(list_instance) - 2:
                formatted_str = formatted_str + f"{cat}, "
            elif index == len(list_instance) - 2:
                formatted_str = formatted_str + f"{cat} & "
            else:
                formatted_str = formatted_str + f"{cat}"
        return formatted_str

    # Running the formatting functions
    num_str = num_format(percentage)
    cat_str = str_format(categories)

    # Setting graph framework
    fig = plotly_ex.bar(
        temp_df,
        x=feature,
        y="Count",
        color="Churn",
        title=f"Churn rate by {feature}",
        barmode="group",
        color_discrete_sequence=["blue", "red"],
    )
    fig.add_annotation(
        text=f"Value count of distribution of {cat_str} are<br>{num_str} percentage respectively.",
        align="left",
        showarrow=False,
        xref="paper",
        yref="paper",
        x=1.4,
        y=1.3,
        bordercolor="black",
        borderwidth=1,
    )
    fig.update_layout(
        # margin space for the annotations on the right
        margin=dict(r=400),
    )

    return fig.show()

In [ ]:
# Gender feature plot
bar_chart("gender")
# SeniorCitizen feature plot
dts_dtframe.loc[
    dts_dtframe.SeniorCitizen == 0, "SeniorCitizen"
] = "No"  # convert 0 to No in all data instances
dts_dtframe.loc[
    dts_dtframe.SeniorCitizen == 1, "SeniorCitizen"
] = "Yes"  # convert 1 to Yes in all data instances
bar_chart("SeniorCitizen")
# Partner feature plot
bar_chart("Partner")
# Dependents feature plot
bar_chart("Dependents")

***
**Demographic analysis Insight**: 
Gender and partner are even distributed with approximate percentage values. The difference in churn is slightly higher in females but the diffreence is negligible. There is a higher proportion of churn amongst younger customers (where SeniorCitizen is No), customers with no partners and customers with no dependents. These analysis on demographic section of data highlights on-senior citizens with no partners and dependents describe a particular segment of customers that are likely to churn.
***

In [ ]:
bar_chart("PhoneService")
bar_chart("MultipleLines")
bar_chart("InternetService")
bar_chart("OnlineSecurity")
bar_chart("OnlineBackup")
bar_chart("DeviceProtection")
bar_chart("TechSupport")
bar_chart("StreamingTV")
bar_chart("StreamingMovies")

***
**Services that each customer has signed up for Insight**:
These category of features shows significant variations across their values. If a customer does not have a phone service, he/she cannot have multiple lines. About 90.3% of the customers have phone services and have the higher rate to churn. Customers who have Fibre optic as internet service are more likely to churn, this can happen due to high prices, competition, customer service, and many other reasons. Fiber optic service is much more expensive than DSL which may be one of the reasons why customers churn. Customers with  OnlineSecurity ,OnlineBackup ,DeviceProtection and TechSupport  are more unlikely to churn. Streaming service is not predictive for churn as it evenly distributed to yes and no options.
***

In [ ]:
bar_chart("Contract")
bar_chart("PaperlessBilling")
bar_chart("PaymentMethod")

**Payment**:
***
The shorter the contract the higher churn rate as those with longer plans face additional barriers when cancelling prematurely. This clearly explains the motivation for companies to have long-term relationship with their customers. Churn Rate is higher for the customers who opted for paperless billing, About 59.2% of the customers make paperless billing. Customers who pay with electronic check are more likely to churn and this kind of payment is more common than other payment types.
***

### Explore Numeric features

In [ ]:
dts_dtframe.dtypes

In [ ]:
try:
    dts_dtframe["TotalCharges"] = dts_dtframe["TotalCharges"].astype(float)
except ValueError as val_err:
    print(val_err)

In [ ]:
dts_dtframe["TotalCharges"] = pdas.to_numeric(
    dts_dtframe["TotalCharges"], errors="coerce"
)
# Fill the missing values with with the median value
dts_dtframe["TotalCharges"] = dts_dtframe["TotalCharges"].fillna(
    dts_dtframe["TotalCharges"].median()
)

In [ ]:
# Defining the histogram plotting function
def hist(feature):
    group_df = dts_dtframe.groupby([feature, "Churn"]).size().reset_index()
    group_df = group_df.rename(columns={0: "Count"})
    fig = plotly_ex.histogram(
        group_df,
        x=feature,
        y="Count",
        color="Churn",
        marginal="box",
        title=f"Churn rate frequency to {feature} distribution",
        color_discrete_sequence=["blue", "red"],
    )
    fig.show()

In [ ]:
hist("tenure")
hist("MonthlyCharges")
hist("TotalCharges")

***
**Customer account information**: The tenure histogram is rightly skewed and shows that majority of customers has been with the telecom company for just the first few months (0-9 months) and the highest rate of churn is also in that first few months (0-9months). 75% of customers who end up leaving Telcom company  do so within their first 30 months. The monthly charge histogram shows that clients with higher monthly charges have a higher churn rate (This suggests that discounts and promotions can be an enticing reason for customers to stay). The total charge trend is quite depict due to variation in frequency.
Lets bin the numeric features into 3 sections based on quantiles (low, medium and high to get more information from it).
***

In [ ]:
# Create an empty DataFrame
bin_df = pdas.DataFrame()

# Update the binning DataFrame
bin_df["tenure_bins"] = pdas.qcut(
    dts_dtframe["tenure"], q=3, labels=["low", "medium", "high"]
)
bin_df["MonthlyCharges_bins"] = pdas.qcut(
    dts_dtframe["MonthlyCharges"], q=3, labels=["low", "medium", "high"]
)
bin_df["TotalCharges_bins"] = pdas.qcut(
    dts_dtframe["TotalCharges"], q=3, labels=["low", "medium", "high"]
)
bin_df["Churn"] = dts_dtframe["Churn"]

# Plot the bar chart of the binned variables
bar_chart("tenure_bins", bin_df)
bar_chart("MonthlyCharges_bins", bin_df)
bar_chart("TotalCharges_bins", bin_df)

***
Based on binning, the low tenure and high monthly charge bins have higher churn rates as supported with the previous analysis. While the low Total charge bin has a higher churn rate. 
***

### Data preprocessing

In [ ]:
# The customerID column is not useful as the feature us used for identification of customers.
dts_dtframe.drop(["customerID"], axis=1, inplace=True)

# Encode categorical features
# Defining the map function
def binary_map(feature):
    return feature.map({"Yes": 1, "No": 0})


# Encoding target feature
dts_dtframe["Churn"] = dts_dtframe[["Churn"]].apply(binary_map)

# Encoding gender category
dts_dtframe["gender"] = dts_dtframe["gender"].map({"Male": 1, "Female": 0})

# Encoding other binary category
binary_list = [
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "PhoneService",
    "PaperlessBilling",
]
dts_dtframe[binary_list] = dts_dtframe[binary_list].apply(binary_map)

# Encoding the other categoric features with more than two categories
dts_dtframe = pdas.get_dummies(dts_dtframe, drop_first=True)

In [ ]:
# Checking the correlation between features
corr = dts_dtframe.corr()

fig = plotly_ex.imshow(corr, width=1000, height=1000)
fig.show()

Correlation is a statistical term is a measure on linear relationship with two variables. Features with high correlation are more linearly dependent and have almost the same effect on the dependent variable. So when two features have a high correlation, we can drop one of the two features.

In [ ]:
import statsmodels.api as sm_api
import statsmodels.formula.api as sm_f_api

# Change variable name seperators to '_'
all_columns = [
    column.replace(" ", "_").replace("(", "_").replace(")", "_").replace("-", "_")
    for column in dts_dtframe.columns
]

# Effect the change to the DataFrame column names
dts_dtframe.columns = all_columns

# Prepare it for the GLM formula
glm_columns = [e for e in all_columns if e not in ["customerID", "Churn"]]
glm_columns = " + ".join(map(str, glm_columns))

# Fiting it to the Generalized Linear Model
glm_model = sm_f_api.glm(
    formula=f"Churn ~ {glm_columns}",
    data=dts_dtframe,
    family=sm_api.families.Binomial(),
)
res = glm_model.fit()
print(res.summary())

In [ ]:
npy.exp(res.params)

In [ ]:
# Feature scaling
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
dts_dtframe["tenure"] = sc.fit_transform(dts_dtframe[["tenure"]])
dts_dtframe["MonthlyCharges"] = sc.fit_transform(dts_dtframe[["MonthlyCharges"]])
dts_dtframe["TotalCharges"] = sc.fit_transform(dts_dtframe[["TotalCharges"]])

#### Creating a baseline model

In [ ]:
# Import Machine learning algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

# Import metric for performance evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Split data into train and test sets
from sklearn.model_selection import train_test_split

X = dts_dtframe.drop("Churn", axis=1)
y = dts_dtframe["Churn"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=50
)  

In [ ]:
def modeling(alg, alg_name, params={}):
    model = alg(
        **params
    )  # Instantiating the algorithm class and unpacking parameters if any
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Performance evaluation
    def print_scores(alg, y_true, y_pred):
        print(alg_name)
        acc_score = accuracy_score(y_true, y_pred)
        print("accuracy: ", acc_score)
        pre_score = precision_score(y_true, y_pred)
        print("precision: ", pre_score)
        rec_score = recall_score(y_true, y_pred)
        print("recall: ", rec_score)
        f_score = f1_score(y_true, y_pred, average="weighted")
        print("f1_score: ", f_score)

    print_scores(alg, y_test, y_pred)
    return model

In [ ]:
# Running logistic regression model
log_model = modeling(LogisticRegression, "Logistic Regression")

In [ ]:
# Feature selection to improve model building
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

log = LogisticRegression()
rfecv = RFECV(
    estimator=log,
    cv=StratifiedKFold(10, random_state=50, shuffle=True),
    scoring="accuracy",
)
rfecv.fit(X, y)

In [ ]:
matplot_plt.figure(figsize=(8, 6))
matplot_plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
matplot_plt.grid()
matplot_plt.xticks(range(1, X.shape[1] + 1))
matplot_plt.xlabel("Number of Selected Features")
matplot_plt.ylabel("CV Score")
matplot_plt.title("Recursive Feature Elimination (RFE)")
matplot_plt.show()

print("The optimal number of features: {}".format(rfecv.n_features_))

In [ ]:
# Saving DataFrame with optimal features
X_rfe = X.iloc[:, rfecv.support_]

# Overview of the optimal features in comparison with the intial DataFrame
print('"X" dimension: {}'.format(X.shape))
print('"X" column list:', X.columns.tolist())
print('"X_rfe" dimension: {}'.format(X_rfe.shape))
print('"X_rfe" column list:', X_rfe.columns.tolist())

In [ ]:
# Splitting data with optimal features
X_train, X_test, y_train, y_test = train_test_split(
    X_rfe, y, test_size=0.3, random_state=50
) 

In [ ]:
# Running logistic regression model
log_model = modeling(LogisticRegression, "Logistic Regression Classification")

Trying other machine learning algorithms:

In [ ]:
# SVC
svc_model = modeling(SVC, "SVC Classification")

In [ ]:
# Random forest
rf_model = modeling(RandomForestClassifier, "Random Forest Classification")

In [ ]:
# Decision tree
dt_model = modeling(DecisionTreeClassifier, "Decision Tree Classification")

In [ ]:
# Naive bayes
nb_model = modeling(GaussianNB, "Naive Bayes Classification")

In [ ]:
# Improve best model by hyperparameter tuning
# Define model
model = LogisticRegression()

# Define evaluation
from sklearn.model_selection import RepeatedStratifiedKFold

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# Define search space
from scipy.stats import loguniform

space = dict()
space["solver"] = ["newton-cg", "lbfgs", "liblinear"]
space["penalty"] = ["none", "l1", "l2", "elasticnet"]
space["C"] = loguniform(1e-5, 1000)

# Define search
from sklearn.model_selection import RandomizedSearchCV

search = RandomizedSearchCV(
    model, space, n_iter=500, scoring="accuracy", n_jobs=-1, cv=cv, random_state=1
)

# Execute search
result = search.fit(X_rfe, y)
# Summarize result
print("Best Score: %s" % result.best_score_)
print("Best Hyperparameters: %s" % result.best_params_)

In [ ]:
params = result.best_params_
params

In [ ]:
# Improving the Logistic Regression model
log_model = modeling(
    LogisticRegression, "Logistic Regression Classification", params=params
)

In [ ]:
# Save the model to device
filename = "model.sav"
joblib.dump(log_model, filename)